# Auto Router

In [ ]:
#|default_exp game.engine.autorouter

In [ ]:
# | hide
%load_ext lab_black

In [ ]:
# | export
from rl18xx.game.engine.core import GameError
from rl18xx.game.engine.graph import Route
import time

In [ ]:
# | export
class AutoRouter:
    def __init__(self, game, flash=None):
        self.game = game
        self.next_hexside_bit = 0
        self.flash = flash

    def compute(self, corporation, **opts):
        static = opts.get("routes", [])
        path_timeout = opts.get("path_timeout", 30)
        route_timeout = opts.get("route_timeout", 10)
        route_limit = opts.get("route_limit", 10000)

        connections = {}
        trains = sorted(
            self.game.route_trains(corporation),
            key=lambda train: train.price,
            reverse=True,
        )

        graph = self.game.graph_for_entity(corporation)
        nodes = sorted(
            graph.connected_nodes(corporation).keys(),
            key=lambda node: (
                0 if node.tokened_by(corporation) else 1,
                0 if node.offboard else 1,
                -max(node.route_revenue(self.game.phase, train) for train in trains),
            ),
        )

        path_walk_timed_out = False
        now = time.time()

        skip_paths = {path for route in static for path in route.paths}
        skip_trains = {route.train for route in static for route in route.train}
        trains = [train for train in trains if train not in skip_trains]

        train_routes = {train: [] for train in trains}
        hexside_bits = {}
        self.next_hexside_bit = 0

        def set_bit(bitfield, bit):
            entry = bit // 32
            mask = 1 << (bit % 32)
            add_count = entry + 1 - len(bitfield)
            bitfield.extend([0] * add_count)
            bitfield[entry] |= mask

        def check_edge_and_set(bitfield, hexside_edge, hexside_bits):
            if hexside_edge in hexside_bits:
                set_bit(bitfield, hexside_bits[hexside_edge])
            else:
                hexside_bits[hexside_edge] = self.next_hexside_bit
                set_bit(bitfield, self.next_hexside_bit)
                self.next_hexside_bit += 1

        def check_and_set(bitfield, hexside_left, hexside_right, hexside_bits):
            check_edge_and_set(bitfield, hexside_left, hexside_bits)
            check_edge_and_set(bitfield, hexside_right, hexside_bits)

        def bitfield_from_connection(connection, hexside_bits):
            bitfield = [0]
            for conn in connection:
                paths = conn["chain"]["paths"]
                if (
                    len(paths) == 1
                ):  # special case for tiny intra-tile path like in 18NewEngland
                    hexside_left = paths[0]["nodes"][0]["id"]
                    check_edge_and_set(bitfield, hexside_left, hexside_bits)
                    if len(paths[0]["nodes"]) > 1:
                        hexside_right = paths[0]["nodes"][1]["id"]
                        check_edge_and_set(bitfield, hexside_right, hexside_bits)
                else:
                    for index in range(len(paths) - 1):
                        node1 = paths[index]
                        node2 = paths[index + 1]
                        if len(node1["edges"]) == 1:
                            hexside_left = node1["edges"][0]["id"]
                            hexside_right = node2["edges"][0]["id"]
                            check_and_set(
                                bitfield, hexside_left, hexside_right, hexside_bits
                            )
                        elif len(node1["edges"]) == 2:
                            hexside_left = node1["edges"][0]["id"]
                            hexside_right = node1["edges"][1]["id"]
                            check_and_set(
                                bitfield, hexside_left, hexside_right, hexside_bits
                            )
                            hexside_left = hexside_right
                            hexside_right = node2["edges"][0]["id"]
                            check_and_set(
                                bitfield, hexside_left, hexside_right, hexside_bits
                            )
            return bitfield

        for node in nodes:
            if time.time() - now > path_timeout:
                print("Path timeout reached")
                path_walk_timed_out = True
                break
            else:
                print(
                    f"Path search: {nodes.index(node) + 1} / {len(nodes)} - paths starting from {node.hex.name}"
                )

            walk_corporation = None if graph.no_blocking else corporation

            def complete():
                chains.append({"nodes": [left, right], "paths": chain})
                nonlocal last_left, last_right
                last_left, last_right = left, right
                left, right = None, None
                chain.clear()

            def assign(a, b):
                nonlocal left, right
                if a and b:
                    if a == last_left or b == last_right:
                        left, right = b, a
                    else:
                        left, right = a, b
                    complete()
                elif not left:
                    left = a or b
                elif not right:
                    right = a or b
                    complete()

            chain = []
            chains = []
            left = None
            right = None
            last_left = None
            last_right = None

            paths = node.walk(
                corporation=walk_corporation,
                skip_paths=skip_paths,
            )

            for path in paths:
                chain.append(path)
                a, b = path.nodes
                assign(a, b)

            if not chains:
                if left:
                    chains.append({"nodes": [left, None], "paths": []})
                continue

            id = tuple(sorted(chain for chain in chains for path in chain["paths"]))
            if id in connections:
                continue

            connections[id] = chains

            path_abort = {train: True for train in trains}

            for train in trains:
                try:
                    route = Route(
                        self.game,
                        self.game.phase,
                        train,
                        connection_data=connections[id],
                        bitfield=bitfield_from_connection(
                            connections[id], hexside_bits
                        ),
                    )
                    route.routes = [route]
                    route.revenue(suppress_check_other=True)
                    train_routes[train].append(route)
                except RouteTooLong:
                    path_abort.pop(train, None)
                except ReusesCity:
                    path_abort.clear()
                except (NoToken, RouteTooShort, GameError):
                    pass

            if not path_abort:
                break

        print(
            f"Evaluated {len(connections)} paths, found {self.next_hexside_bit} unique hexsides, and found valid routes "
            f"{', '.join(f'{train.name}:{len(routes)}' for train, routes in train_routes.items())} in: {time.time() - now}"
        )

        for route in static:
            route.bitfield = bitfield_from_connection(
                route.connection_data, hexside_bits
            )
            train_routes[route.train] = [route]

        for train, routes in train_routes.items():
            train_routes[train] = sorted(
                routes, key=lambda route: route.revenue, reverse=True
            )[:route_limit]

        sorted_routes = list(train_routes.values())

        limit = 1
        for routes in sorted_routes:
            limit *= len(routes)

        print(
            f"Finding route combos of best {', '.join(f'{train.name}:{len(routes)}' for train, routes in train_routes.items())} "
            f"routes with depth {limit}"
        )

        now = time.time()
        possibilities = self.evaluate_combos(sorted_routes, route_timeout)

        if path_walk_timed_out:
            self.flash("Auto route path walk failed to complete (PATH TIMEOUT)")
        elif time.time() - now > route_timeout:
            self.flash("Auto route selection failed to complete (ROUTE TIMEOUT)")

        max_routes = max(
            possibilities,
            key=lambda routes: self.game.routes_revenue(routes),
            default=[],
        )

        for route in max_routes:
            route.routes = max_routes

    def evaluate_combos(sorted_routes, route_timeout):
        possibilities = []
        possibilities_count = 0
        conflicts = 0
        now = time.time()

        combos = []
        counter = 0
        max_revenue = 0

        js_now = int(time.time() * 1000)
        js_route_timeout = route_timeout * 1000

        js_sorted_routes = []
        limit = 1

        for rb_routes in sorted_routes:
            js_routes = []

            limit *= len(rb_routes)

            for rb_route in rb_routes:
                js_routes.append(
                    {
                        "route": rb_route,
                        "bitfield": rb_route.bitfield,
                        "revenue": rb_route.revenue,
                    }
                )

            js_sorted_routes.append(js_routes)

        old_limit = limit

        for r in range(len(js_sorted_routes[0])):
            route = js_sorted_routes[0][r]
            counter += 1
            combo = {"revenue": route["revenue"], "routes": [route]}
            combos.append(combo)

            if is_valid_combo(combo):
                possibilities_count += 1

                if combo["revenue"] >= max_revenue:
                    if combo["revenue"] > max_revenue:
                        possibilities = []
                        max_revenue = combo["revenue"]
                    possibilities.append(combo)

        continue_looking = True

        for train in range(1, len(js_sorted_routes)):
            limit = len(combos)

            for remaining in range(train, len(js_sorted_routes)):
                limit *= len(js_sorted_routes[remaining])

            if limit != old_limit:
                print(
                    f"Adjusting depth to {limit} because first {train} trains only had {len(combos)} valid combos"
                )
                old_limit = limit

            new_combos = []

            for rt in range(len(js_sorted_routes[train])):
                route = js_sorted_routes[train][rt]

                for c in range(len(combos)):
                    combo = combos[c]
                    counter += 1

                    if counter % 1_000_000 == 0:
                        print(f"{counter} / {limit}")

                        if int(time.time() * 1000) - js_now > js_route_timeout:
                            print("Route timeout reached")
                            continue_looking = False
                            break

                    if route_bitfield_conflicts(combo, route):
                        conflicts += 1
                    else:
                        newcombo = {
                            "revenue": combo["revenue"],
                            "routes": combo["routes"][:] + [route],
                        }
                        newcombo["revenue"] += route["revenue"]
                        new_combos.append(newcombo)

                        if is_valid_combo(newcombo):
                            possibilities_count += 1

                            if newcombo["revenue"] >= max_revenue:
                                if newcombo["revenue"] > max_revenue:
                                    possibilities = []
                                    max_revenue = newcombo["revenue"]
                                possibilities.append(newcombo)

            for combo in new_combos:
                combos.append(combo)

        for combo in possibilities:
            rb_routes = []

            for route in combo["routes"]:
                rb_routes.append(route["route"])

            possibilities_count.append(rb_routes)

        print(
            f"Found {possibilities_count} possible combos ({len(possibilities)} best) and rejected {conflicts} "
            f"conflicting combos in: {time.time() - now}"
        )

        return possibilities_count

    def is_valid_combo(cb):
        # Implement your validation logic here.
        # Return True if the combo is valid, otherwise return False.
        pass

    def route_bitfield_conflicts(combo, testroute):
        # Implement your bitfield conflict checking logic here.
        # Return True if there is a conflict, otherwise return False.
        pass

    def is_valid_combo(cb):
        rb_rts = []

        for rt in cb["routes"]:
            rb_rts.append(rt["route"])
            rt["route"].routes = rb_rts

        try:
            for rt in cb["routes"]:
                rt["route"].check_other()
            return True
        except Exception as err:
            return False

    def route_bitfield_conflicts(combo, testroute):
        for cr in combo["routes"]:
            index = min(len(cr["bitfield"]), len(testroute["bitfield"])) - 1

            while index >= 0:
                if (cr["bitfield"][index] & testroute["bitfield"][index]) != 0:
                    return True
                index -= 1

        return False